In [ ]:
data = [dict(x=1.0, y=1.0, cluster_id=1, distance=4.0),
        dict(x=1.0, y=1.0, cluster_id=2, distance=6.3),
        dict(x=2.0, y=1.0, cluster_id=1, distance=3.0),
        dict(x=2.0, y=1.0, cluster_id=2, distance=2.1),]

field_x = StructField('x', FloatType(), True)
field_y = StructField('y', FloatType(), True)
field_id = StructField('cluster_id', IntegerType(), True)
field_dist = StructField('distance', FloatType(), True)

schema = StructType([field_x, field_y, field_id, field_dist])
distances = spark.createDataFrame(data, schema)
distances.createOrReplaceTempView('distances')

query = '''
    WITH closest AS (
        SELECT
            x, y,
            MIN(distance) AS min_dist
        FROM distances
        GROUP BY x, y )

    SELECT
        sdf1.x AS x, 
        sdf1.y AS y, 
        sdf1.cluster_id AS cluster_id
    FROM distances AS sdf1
    INNER JOIN closest AS sdf2
    ON sdf1.x = sdf2.x
    AND sdf1.y = sdf2.y
    AND sdf1.distance = sdf2.min_dist
'''    

points = spark.sql(query)
points.show()